## Read in bindingsite & Ensembl data

In [3]:
%load_ext autoreload
%autoreload 2
from helper_fcts import *
import sqlite3
import os
import pandas as pd
from pathlib import Path
import numpy as np
test = False
path = Path('data')
tcga_path = Path(path/'KICH')
path_tarp = Path(path/'tarp-bs') if test else Path('../data')
path_input = Path(path/'tarp-bs/cdna') if test else Path('../data/all_cdna_subsets')

In [21]:
#test bindingsite big
bs = pd.read_parquet(path/'bs_chrom_pos1.parquet')
bs = bs[['miRNA','mRNA','chrom_bs_start','chrom_bs_end']]
bs['miRNA'] = pd.Categorical(bs.miRNA)
bs['mRNA'] = pd.Categorical(bs.mRNA)
bs['bs_id'] = bs.index
df_gene = pd.read_parquet(path/'ensembl_transcript_info.parquet')
bs = bs.merge(df_gene, left_on='mRNA', right_on='ensembl_transcript_id', how='left')
del bs['ensembl_transcript_id']
bs['Strand'] = '-'
bs.loc[bs['strand'] == '1', 'Strand'] = '+'
del bs['strand']
bs['Strand'] = pd.Categorical(bs.Strand)
#bs[bs.isnull().values] #problem: 20 572 214 / 99 999 996 chromosome NaN -> cause they spanned more than 1 exon
bs = bs.dropna()

In [11]:
exon_counts = pd.read_parquet(path/'KICH'/'temp_exon_counts.parquet') 
exon_counts['temp'] = '-'
exon_counts.loc[exon_counts['strand'] == '1', 'temp'] = '+'
exon_counts['strand'] = exon_counts['temp'] 
exon_counts['strand'] = pd.Categorical(exon_counts.strand)
exon_counts = exon_counts.drop(['temp','ensembl_transcript_id','exon_count','exon_id'],axis=1)
exon_counts['e_id'] = exon_counts.index

In [4]:
mirna_counts = pd.read_csv(tcga_path/'miRNA_HiSeq_gene', delimiter='	')
caseids = list(set(mirna_counts.columns).intersection(list(exon_counts.columns)))

In [26]:
def sqlite_join_1():
    conn = sqlite3.connect('speed.db') #Make the database in memory
    c = conn.cursor()
    #write the tables
    bs.to_sql('counts', conn, index=False, dtype={"bs_id": 'INTEGER'})
    exon_counts[['chromosome_name','strand','chrom_exon_start','chrom_exon_end','patient_id','exon_count','exon_id']].to_sql('exon_counts', conn, dtype={caseid : 'INTEGER' for caseid in caseids}.update({"chrom_exon_start": 'INTEGER', "chrom_exon_end": 'INTEGER'}))
    qry = '''
        CREATE TABLE n_counts AS
        select
            exon_id, bs_id, miRNA, patient_id, exon_count
        from
            exon_counts inner join counts
            on
            counts.chromosome_name = exon_counts.chromosome_name and counts.strand = exon_counts.strand and counts.chrom_bs_start >= exon_counts.chrom_exon_start and counts.chrom_bs_end <= exon_counts.chrom_exon_end
        '''
    print('start query')
    return pd.read_sql_query(qry,conn)
    #only returns bs with minimum 1 exon_id cause inner join (else left join)
    
def sqlite_join_2():
    conn = sqlite3.connect('speed.db') #Make the database in memory
    c = conn.cursor()
    #write the tables
    bs.to_sql('counts', conn, index=False, dtype={"bs_id": 'INTEGER'})
    exon_counts[['ensembl_transcript_id','chrom_exon_start','chrom_exon_end','patient_id','exon_count','exon_id']].to_sql('exon_counts', conn, dtype={caseid : 'INTEGER' for caseid in caseids}.update({"chrom_exon_start": 'INTEGER', "chrom_exon_end": 'INTEGER'}))
    qry = '''
        CREATE TABLE n_counts AS
        select
            exon_id, bs_id, miRNA, patient_id, exon_count
        from
            exon_counts inner join counts
            on
            counts.mRNA = exon_counts.ensembl_transcript_id and counts.chrom_bs_start >= exon_counts.chrom_exon_start and counts.chrom_bs_end <= exon_counts.chrom_exon_end
        '''
    print('start query')
    return pd.read_sql_query(qry,conn)

In [ ]:
%timeit new_bs = sqlite_join_1() 
#30 - 56 GB - 34 
#2.18 - 2.24 (start query) - > 1 hour

start query


In [ ]:
new_bs.to_parquet(path/'bs_chrom_exon_pos1.parquet')


In [6]:
%memit new_bs = sqlite_join()


UsageError: Line magic function `%memit` not found.


In [7]:
bs = parse_tarp_bs(path/'tarp-bs'/'input_miRNA.fasta_cdna6247.fasta.bp')
bs = bs[['miRNA','mRNA','bs_start','bs_end']]
bs['miRNA'] = pd.Categorical(bs.miRNA)
bs['mRNA'] = pd.Categorical(bs.mRNA)
bs['bs_id'] = bs.index
#bs.info() # 2.6GB
df_gene = pd.read_parquet(path/'ensembl_transcript_info.parquet')
bs = bs.merge(df_gene, left_on='mRNA', right_on='ensembl_transcript_id', how='left')
bs['mRNA'] = pd.Categorical(bs.mRNA)
del bs['ensembl_transcript_id']
bs['Strand'] = '-'
bs.loc[bs['strand'] == '1', 'Strand'] = '+'
del bs['strand']
bs['Strand'] = pd.Categorical(bs.Strand)

exon_counts = pd.read_parquet(path/'KICH'/'temp_exon_counts.parquet') 
exon_counts['temp'] = '-'
exon_counts.loc[exon_counts['strand'] == '1', 'temp'] = '+'
exon_counts['strand'] = exon_counts['temp'] 
exon_counts['strand'] = pd.Categorical(exon_counts.strand)
exon_counts = exon_counts.drop(['temp','ensembl_transcript_id','exon_id'],axis=1)
exon_counts

,chromosome_name,strand,chrom_exon_start,chrom_exon_end,patient_id,exon_count
0,3,-,51973965,51974630,TCGA-KN-8419-01,2.1156
1,1,-,215728030,215728384,TCGA-KN-8419-01,0.0000
2,17,-,51191601,51193016,TCGA-KN-8419-01,0.3884
3,4,-,146292929,146292980,TCGA-KN-8419-01,2.5290
4,1,+,15747546,15747980,TCGA-KN-8419-01,0.0000
...,...,...,...,...,...,...
30325745,3,+,51978259,51978374,TCGA-KL-8327-01,4.1126
30325746,6,-,43357126,43357765,TCGA-KL-8327-01,0.2725
30325747,17,-,36168630,36168695,TCGA-KL-8327-01,1.9102
30325748,1,+,120833602,120833774,TCGA-KL-8327-01,1.1468


In [9]:
def bedtool_join():    
    import pybedtools
    from pybedtools import BedTool
    print('convert data to BedTool')
    bs_bed = bs[['chromosome_name', 'bs_start', 'bs_end', 'bs_id','miRNA','Strand']].copy()
    exon_bed = exon_counts[['chromosome_name', 'chrom_exon_start', 'chrom_exon_end', 'patient_id','exon_count','strand']].copy()
    bs_bed = pybedtools.BedTool.from_dataframe(bs_bed, header=['chrom','start', 'end', 'name', 'score','strand'])
    exon_bed = pybedtools.BedTool.from_dataframe(exon_bed, header=['chrom','start', 'end', 'name', 'score','strand'])
    print('start join')
    return bs_bed.intersect(exon_bed)#, loj=True)

In [16]:
#new_bs = bedtool_join() 
jo = pd.read_table(new_bs.fn, names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
jo

,chrom,start,stop,name,score,strand


In [ ]:
new_bs = bedtool_join().to_dataframe()
new_bs.to_parquet(path/'mini_bs_exon_bedtool.parquet')
#34 - 
#2.39-2.41 to convert to bed files
#3.20-

In [4]:
#small bs #TODO chromosoooomeeee
bs = parse_tarp_bs(path/'tarp-bs'/'input_miRNA.fasta_cdna6247.fasta.bp')
bs['bs_id'] = bs.index
df_gene = pd.read_parquet(path/'ensembl_transcript_info.parquet')
bs = bs.merge(df_gene, left_on='mRNA', right_on='ensembl_transcript_id', how='left')
bs = bs[['miRNA','mRNA','bs_start','bs_end','bs_id','chromosome_name','strand']]
new_row = {'miRNA':'hsa-let-7a-2-3p', 'mRNA':'ENST00000439513', 'bs_start':1517259.0, 'bs_end':1517304.0, 'bs_id':73193,'chromosome_name': '1','strand': '1'}
bs = bs.append(new_row, ignore_index=True)
bs['Strand'] = '-'
bs.loc[bs['strand'] == '1', 'Strand'] = '+'
del bs['strand']
bs['chromosome_name'] = pd.Categorical(bs.chromosome_name)
bs['Strand'] = pd.Categorical(bs.Strand)
#bs[bs.isnull().values] #problem: 20 572 214 / 99 999 996 chromosome NaN -> cause they spanned more than 1 exon
bs = bs.dropna()

exon_counts = pd.read_parquet(path/'KICH'/'temp_exon_counts.parquet') 
exon_counts['temp'] = '-'
exon_counts.loc[exon_counts['strand'] == '1', 'temp'] = '+'
exon_counts['strand'] = exon_counts['temp'] 
exon_counts['strand'] = pd.Categorical(exon_counts.strand)
exon_counts = exon_counts.drop(['temp','ensembl_transcript_id','exon_id'],axis=1)#'exon_count'
#exon_counts = exon_counts.dropna() #no NA

In [35]:
exon_counts

,chromosome_name,strand,chrom_exon_start,chrom_exon_end,patient_id,exon_count
0,3,-,51973965,51974630,TCGA-KN-8419-01,2.1156
1,1,-,215728030,215728384,TCGA-KN-8419-01,0.0000
2,17,-,51191601,51193016,TCGA-KN-8419-01,0.3884
3,4,-,146292929,146292980,TCGA-KN-8419-01,2.5290
4,1,+,15747546,15747980,TCGA-KN-8419-01,0.0000
...,...,...,...,...,...,...
30325745,3,+,51978259,51978374,TCGA-KL-8327-01,4.1126
30325746,6,-,43357126,43357765,TCGA-KL-8327-01,0.2725
30325747,17,-,36168630,36168695,TCGA-KL-8327-01,1.9102
30325748,1,+,120833602,120833774,TCGA-KL-8327-01,1.1468


In [36]:
def pyranges_join():
    import pyranges as pr
    #TODO for parallel ray 
    
    # Add column names to the dfs so PyRanges knows which 
    # represent Chromosomes, Starts, Ends and optionally Strand
    ranges_bs = bs.copy()
    ranges_bs.columns = "miRNA mRNA Start End bs_id Chromosome Strand".split()
    ranges_exon = exon_counts.copy()
    ranges_exon.columns = "Chromosome Strand Start End patient_id exon_count".split()
    import datetime
    now = datetime.datetime.now()
    hour = int(now.strftime('%H'))
    minu = int(now.strftime('%M'))
    # create PyRanges-objects from the dfs
    gr1, gr2 = pr.PyRanges(ranges_bs), pr.PyRanges(ranges_exon)
    print('start join')
    # intersect the two
    gr = gr1.join(gr2)
    #gr = gr1.intersect(gr2)#, nb_cpu=2)#, how='containment') returns all overlapping bs

    now = datetime.datetime.now()
    hour2 = int(now.strftime('%H'))
    minu2 = int(now.strftime('%M'))
    print(hour2,':',minu2,'-',hour,':',minu,'=',hour2-hour,':',minu2-minu)
    # get the result as a pandas df
    return gr.df
new_bs = pyranges_join()

start join
13 : 28 - 13 : 27 = 0 : 1


,miRNA,mRNA,Start,End,bs_id,Chromosome,Strand,Strand_b,Start_b,End_b,patient_id,exon_count
0,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KN-8419-01,2.9478
1,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KL-8346-01,2.3182
2,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KN-8422-01,2.3118
3,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KN-8431-11,2.9785
4,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KN-8430-11,3.1038
...,...,...,...,...,...,...,...,...,...,...,...,...
86,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KO-8408-01,2.5630
87,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KM-8443-01,2.3830
88,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KM-8442-01,2.0297
89,hsa-let-7a-2-3p,ENST00000439513,1517259,1517304,73193,1,+,+,1517167,1517412,TCGA-KL-8332-11,1.8481


In [40]:
new_bs = pyranges_join()
new_bs.to_parquet(path/'bs_pos1_chrom_exon_pyranges.parquet')
new_bs = new_bs.df.drop(['mRNA','Start','End','Chromosome','Strand','Strand_b','Start_b','End_b'],axis=1)
new_bs.to_parquet(path/'bs_pos1_chrom_exon_subset_pyranges.parquet')
new_bs

,miRNA,bs_id,patient_id,exon_count
0,hsa-let-7a-2-3p,73193,TCGA-KN-8419-01,2.9478
1,hsa-let-7a-2-3p,73193,TCGA-KL-8346-01,2.3182
2,hsa-let-7a-2-3p,73193,TCGA-KN-8422-01,2.3118
3,hsa-let-7a-2-3p,73193,TCGA-KN-8431-11,2.9785
4,hsa-let-7a-2-3p,73193,TCGA-KN-8430-11,3.1038
...,...,...,...,...
86,hsa-let-7a-2-3p,73193,TCGA-KO-8408-01,2.5630
87,hsa-let-7a-2-3p,73193,TCGA-KM-8443-01,2.3830
88,hsa-let-7a-2-3p,73193,TCGA-KM-8442-01,2.0297
89,hsa-let-7a-2-3p,73193,TCGA-KL-8332-11,1.8481
